In [58]:
import subprocess
import os
from dotenv import load_dotenv
import json

load_dotenv('../.env')

output = subprocess.run(['node', '../nodejs/roundbench.js'], stdout=subprocess.PIPE, text=True)
json.loads(output.stdout)

{'start': [261997, 425284875],
 'place_response': [261999, 162892625],
 'place_json_parsed': [261999, 168488500],
 'ws_place': [261999, 626357916],
 'ws_cancel_parsed': [261999, 173414666],
 'cancel_response': [261999, 625983833],
 'cancel_json_parsed': [261999, 626013083],
 'ws_place_parsed': [261999, 626370833],
 'end': [261999, 626388625]}

In [59]:
output = subprocess.run(['../rust/target/release/roundbench'], stdout=subprocess.PIPE, text=True)
json.loads(output.stdout)

{'start': [1686676509, 541262000],
 'place_response': None,
 'place_json_parsed': [1686676510, 71268000],
 'ws_place': None,
 'ws_cancel_parsed': [1686676510, 608973000],
 'cancel_response': None,
 'cancel_json_parsed': [1686676510, 606677000],
 'ws_place_parsed': [1686676510, 72357000],
 'end': None}

In [60]:
import time
import random

def sample_node():
    output = subprocess.run(['node', '../nodejs/roundbench.js'], stdout=subprocess.PIPE, text=True)
    sample = json.loads(output.stdout)
    sample['platform'] = 'NodeJS'
    return sample

def sample_rust():
    output = subprocess.run(['../rust/target/release/roundbench'], stdout=subprocess.PIPE, text=True)
    sample = json.loads(output.stdout)
    sample['platform'] = 'Rust'
    return sample

int_to_sampler = {
    1: sample_node,
    2: sample_rust
}

samples = []
for i in range(1000):
    samples.append(int_to_sampler[random.randint(1, 2)]())
    time.sleep(1)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [61]:
import pandas as pd

df = pd.DataFrame(samples)
df.to_csv('samples.csv')

In [101]:
def to_ns(x):
    return x[0] * 10**9 + x[1]
df['start_ns'] = df['start'].apply(to_ns)
df['ws_place_parsed_ns'] = df['ws_place_parsed'].apply(to_ns)
df['ws_cancel_parsed_ns'] = df['ws_cancel_parsed'].apply(to_ns)
df['duration_place_cancel_ws_ns'] = df['ws_cancel_parsed_ns'] - df['ws_place_parsed_ns']
df['duration_ns'] = df['ws_cancel_parsed_ns'] - df['start_ns']
df.head()


,start,place_response,place_json_parsed,ws_place,ws_cancel_parsed,cancel_response,cancel_json_parsed,ws_place_parsed,end,platform,start_ns,ws_cancel_parsed_ns,duration_ns,ws_place_parsed_ns,duration_place_cancel_ws_ns
0,"[1686676511, 726042000]",None,"[1686676512, 263118000]",None,"[1686676512, 904373000]",None,"[1686676512, 902902000]","[1686676512, 377682000]",None,Rust,1686676511726042000,1686676512904373000,1178331000,1686676512377682000,526691000
1,"[262005, 157720791]","[262006, 352102291]","[262006, 352189583]","[262006, 675727125]","[262006, 358377208]","[262006, 675546791]","[262006, 675605791]","[262006, 675863041]","[262006, 675918458]",NodeJS,262005157720791,262006358377208,1200656417,262006675863041,-317485833
2,"[1686676517, 603824000]",None,"[1686676518, 129744000]",None,"[1686676518, 663668000]",None,"[1686676518, 660783000]","[1686676518, 129646000]",None,Rust,1686676517603824000,1686676518663668000,1059844000,1686676518129646000,534022000
3,"[262010, 921955583]","[262012, 145020500]","[262012, 145061333]","[262012, 478089958]","[262012, 145917041]",NaN,NaN,"[262012, 478148791]","[262012, 478263625]",NodeJS,262010921955583,262012145917041,1223961458,262012478148791,-332231750
4,"[262013, 537913708]","[262014, 725275291]","[262014, 725320875]","[262015, 271476333]","[262014, 732884125]","[262015, 270469875]","[262015, 270575000]","[262015, 271546750]","[262015, 271602833]",NodeJS,262013537913708,262014732884125,1194970417,262015271546750,-538662625


In [128]:
dfc = df.copy()
# dfc['wsc'] = dfc[dfc['platform'] == 'NodeJS']['ws_place_parsed'].apply(to_ns)
# dfc['wsc'] = dfc[dfc['platform'] == 'Rust']['ws_cancel_parsed'].apply(to_ns)
# dfc['wsc']
for r in dfc.iterrows():
    i = r[0]
    if r[1]['platform'] == 'NodeJS':
        df.at[i, 'duration_corrected'] = -r[1]['duration_place_cancel_ws_ns']
    else:
        df.at[i, 'duration_corrected'] = r[1]['duration_place_cancel_ws_ns']
df.head()

,start,place_response,place_json_parsed,ws_place,ws_cancel_parsed,cancel_response,cancel_json_parsed,ws_place_parsed,end,platform,start_ns,ws_cancel_parsed_ns,duration_ns,ws_place_parsed_ns,duration_place_cancel_ws_ns,duration_corrected
0,"[1686676511, 726042000]",None,"[1686676512, 263118000]",None,"[1686676512, 904373000]",None,"[1686676512, 902902000]","[1686676512, 377682000]",None,Rust,1686676511726042000,1686676512904373000,1178331000,1686676512377682000,526691000,526691000.0
1,"[262005, 157720791]","[262006, 352102291]","[262006, 352189583]","[262006, 675727125]","[262006, 358377208]","[262006, 675546791]","[262006, 675605791]","[262006, 675863041]","[262006, 675918458]",NodeJS,262005157720791,262006358377208,1200656417,262006675863041,-317485833,317485833.0
2,"[1686676517, 603824000]",None,"[1686676518, 129744000]",None,"[1686676518, 663668000]",None,"[1686676518, 660783000]","[1686676518, 129646000]",None,Rust,1686676517603824000,1686676518663668000,1059844000,1686676518129646000,534022000,534022000.0
3,"[262010, 921955583]","[262012, 145020500]","[262012, 145061333]","[262012, 478089958]","[262012, 145917041]",NaN,NaN,"[262012, 478148791]","[262012, 478263625]",NodeJS,262010921955583,262012145917041,1223961458,262012478148791,-332231750,332231750.0
4,"[262013, 537913708]","[262014, 725275291]","[262014, 725320875]","[262015, 271476333]","[262014, 732884125]","[262015, 270469875]","[262015, 270575000]","[262015, 271546750]","[262015, 271602833]",NodeJS,262013537913708,262014732884125,1194970417,262015271546750,-538662625,538662625.0


In [105]:
# df['duration_ns'].groupby('platform').describe()
df[df['platform'] == 'NodeJS']['duration_place_cancel_ws_ns'].max() / 10**9

-0.308735584

In [109]:
df[df['platform'] == 'Rust']['duration_place_cancel_ws_ns'].mean() / 10**9

0.5402873469387756

In [135]:
dfc['place_json_parsed_ns'] = dfc['place_json_parsed'].apply(to_ns)
dfc['cancel_json_parsed_ns'] = dfc['cancel_json_parsed'].dropna().apply(to_ns)

dfc['place_cancel_res_duration_ns'] = dfc['cancel_json_parsed_ns'] - dfc['place_json_parsed_ns']
# dfc.head

,start,place_response,place_json_parsed,ws_place,ws_cancel_parsed,cancel_response,cancel_json_parsed,ws_place_parsed,end,platform,start_ns,ws_cancel_parsed_ns,duration_ns,ws_place_parsed_ns,duration_place_cancel_ws_ns,duration_corrected,place_json_parsed_ns,cancel_json_parsed_ns,place_cancel_res_duration_ns
0,"[1686676511, 726042000]",None,"[1686676512, 263118000]",None,"[1686676512, 904373000]",None,"[1686676512, 902902000]","[1686676512, 377682000]",None,Rust,1686676511726042000,1686676512904373000,1178331000,1686676512377682000,526691000,526691000.0,1686676512263118000,1.686677e+18,639783936.0
1,"[262005, 157720791]","[262006, 352102291]","[262006, 352189583]","[262006, 675727125]","[262006, 358377208]","[262006, 675546791]","[262006, 675605791]","[262006, 675863041]","[262006, 675918458]",NodeJS,262005157720791,262006358377208,1200656417,262006675863041,-317485833,317485833.0,262006352189583,2.620067e+14,323416208.0
2,"[1686676517, 603824000]",None,"[1686676518, 129744000]",None,"[1686676518, 663668000]",None,"[1686676518, 660783000]","[1686676518, 129646000]",None,Rust,1686676517603824000,1686676518663668000,1059844000,1686676518129646000,534022000,534022000.0,1686676518129744000,1.686677e+18,531039232.0
3,"[262010, 921955583]","[262012, 145020500]","[262012, 145061333]","[262012, 478089958]","[262012, 145917041]",NaN,NaN,"[262012, 478148791]","[262012, 478263625]",NodeJS,262010921955583,262012145917041,1223961458,262012478148791,-332231750,332231750.0,262012145061333,NaN,NaN
4,"[262013, 537913708]","[262014, 725275291]","[262014, 725320875]","[262015, 271476333]","[262014, 732884125]","[262015, 270469875]","[262015, 270575000]","[262015, 271546750]","[262015, 271602833]",NodeJS,262013537913708,262014732884125,1194970417,262015271546750,-538662625,538662625.0,262014725320875,2.620153e+14,545254125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,"[262601, 254588375]","[262602, 490004916]","[262602, 490097250]","[262602, 839440541]","[262602, 507794666]","[262602, 838657750]","[262602, 838713875]","[262602, 839473958]","[262602, 839529041]",NodeJS,262601254588375,262602507794666,1253206291,262602839473958,-331679292,331679292.0,262602490097250,2.626028e+14,348616625.0
204,"[1686677113, 778690000]",None,"[1686677114, 390730000]",None,"[1686677114, 912135000]",None,"[1686677114, 909285000]","[1686677114, 380231000]",None,Rust,1686677113778690000,1686677114912135000,1133445000,1686677114380231000,531904000,531904000.0,1686677114390730000,1.686677e+18,518555136.0
205,"[262607, 182925541]","[262608, 374109750]","[262608, 374196500]","[262608, 688792541]","[262608, 376318291]","[262608, 685938416]","[262608, 686001041]","[262608, 688829666]","[262608, 688896208]",NodeJS,262607182925541,262608376318291,1193392750,262608688829666,-312511375,312511375.0,262608374196500,2.626087e+14,311804541.0
206,"[1686677119, 794924000]",None,"[1686677120, 594874000]",None,"[1686677121, 171774000]",None,None,"[1686677120, 594766000]",None,Rust,1686677119794924000,1686677121171774000,1376850000,1686677120594766000,577008000,577008000.0,1686677120594874000,NaN,NaN


In [137]:
dfc[dfc['platform'] == 'NodeJS']['place_cancel_res_duration_ns'].dropna().describe()

count    1.040000e+02
mean     3.736907e+08
std      1.492841e+08
min      3.085790e+08
25%      3.232918e+08
50%      3.339702e+08
75%      3.603164e+08
max      1.642148e+09
Name: place_cancel_res_duration_ns, dtype: float64

In [138]:
dfc[dfc['platform'] == 'Rust']['place_cancel_res_duration_ns'].dropna().describe()

count    9.100000e+01
mean     5.360290e+08
std      3.695946e+07
min      5.102369e+08
25%      5.173743e+08
50%      5.278039e+08
75%      5.412760e+08
max      8.003658e+08
Name: place_cancel_res_duration_ns, dtype: float64

In [141]:
import pandas as pd

message_df = pd.DataFrame([['a', '1'], ['b', '2'], ['c', '3']], columns=['message', 'keyword'])
suggestion_df = pd.DataFrame([['x', '1'], ['y', '2']], columns=['suggestion', 'keyword'])

the_talent_message_keyword = message_df.iloc[0]['keyword']
keyword_matching_suggestions = suggestion_df[suggestion_df['keyword'] == the_talent_message_keyword]
# suggestion_df[suggestion_df['keyword']isin(message_df)]


'1'